In [1]:
import torch
import matplotlib.pyplot as plt
from tqdm import trange

import pymc as pm
import pytensor as pt
from pymc import PolyaGamma as PG
from pymc import Normal as N

In [7]:
def _compute_mean_a(v, b, theta, w, y):
    S = torch.sum(torch.mul(theta, y-0.5+torch.mul(b, w)))  
    mean = v*S        
    return mean

def _compute_var_a(sigmasq, theta, w):
    S = torch.sum(torch.mul(w, torch.pow(theta, 2))) + 1/sigmasq
    var = 1/S
    return var

def _compute_mean_b(v, a, theta, w, y):
    S = torch.sum(y-0.5-a*torch.mul(theta, w))
    mean = v*S
    return mean

def _compute_var_b(sigmasq, w):
    S = torch.sum(w) + 1/sigmasq
    var = 1/S
    return var

def _compute_mean_t(v, a, b, w, y):
    S = torch.sum(torch.mul(a, y-0.5+torch.mul(b, w)))  
    mean = v*S
    return mean    

def _compute_var_t(sigmasq, a, w):
    S = torch.sum(torch.mul(w, torch.pow(a, 2))) + 1/sigmasq
    #print("S ", S)
    #print(torch.mul(w, torch.pow(a, 2)))
    var = 1/S
    return var

def polyagamma(init_a, init_b, init_t, init_w, y, sigmasq_a, sigmasq_b, sigmasq_t, niter=10000):

    assert init_a.shape == init_b.shape
    assert init_t.shape[0] == y.shape[0]
    assert y.shape == init_w.shape
    
    I = len(init_a)
    P = len(init_t)
    
    # samples
    A = torch.empty(size=(niter, I))
    B = torch.empty(size=(niter, I))
    THETA = torch.empty(size=(niter, P))
    W = torch.empty(size=(niter, P, I))
    
    A[0] = init_a
    B[0] = init_b
    THETA[0] = init_t
    W[0] = init_w

    print(f"Starting Gibbs sampler for Polya-Gamma... \n--------------------------------------------\n")
    for s in trange(niter):

        # sample Ws
        for p in range(P):
            for i in range(I):
                W[s][p][i] = torch.from_numpy(pm.draw(PG.dist(h=1, z=(A[s-1][i]*THETA[s-1][p] - B[s-1][i]))))

        # sample As
        for i in range(I):
            var_a = float(_compute_var_a(sigmasq_a, THETA[s-1], W[s][:,i]))
            mean_a = float(_compute_mean_a(var_a, B[s-1][i], THETA[s-1], W[s][:,i], y[:,i]))
            print(var_a)
            print(mean_a)
            #try:
            A[s][i] = torch.distributions.Normal(mean_a, var_a).sample()
            #except ValueError:
            #    A[s][i] = torch.distributions.Normal(mean_a, var_a+0.001).sample()

        # sample Bs
        for i in range(I):
            var_b = float(_compute_var_b(sigmasq_b, W[s][:,i]))
            mean_b = float(_compute_mean_b(var_b, A[s][i], THETA[s-1], W[s][:,i], y[:,i]))
            B[s][i] = torch.distributions.Normal(mean_b, var_b).sample()

        # sample THETAs
        for p in range(P):
            var_t = float(_compute_var_t(sigmasq_t, W[s][p], A[s]))
            mean_t = float(_compute_mean_t(var_t, A[s], B[s], W[s][p,:], y[p,:]))
            #print("mean: ", mean_t)
            #print("variance: ", var_t)
            #THETA[s][p] = torch.from_numpy(pm.draw(N.dist(mean_t, var_t)))
            THETA[s][p] = torch.distributions.Normal(mean_t, var_t).sample()

    return A, B, THETA, W

In [3]:
# TEST

I = 10   # exam items
P = 100  # pupils
sigmasq_a, sigmasq_b, sigmasq_t = torch.tensor([1.0]), torch.tensor([1.0]), torch.tensor([1.0])  # priors

init_a = torch.zeros(I)  
init_b = torch.zeros(I)  
init_t = torch.zeros(P)
init_W = torch.zeros(P, I)

true_a = torch.tensor([1, 0.9, 0.01, 0.5, 0.7, 0.4, 0.03, 0.9, 0.8, 1])  # items' discriminatory power
true_b = torch.tensor([0.01, 0.9, 1, 0.8, 0.2, 0.3, 0.88, 1, 0.3, 0.5])  # items' difficulty
true_theta = torch.empty(size=(P,)) #torch.tensor([1, 1, 0, 0.5, 0.7, 0.1, 0.3, 0.9, 0.6, 0.7])  # students' skills
true_W = torch.empty(size=(P,I))

# populate theta:
for p in range(P):
    U = torch.distributions.Uniform(torch.tensor([0.0]), torch.tensor([1.0]))
    true_theta[p] = U.sample()

# populate W:
for p in range(P):
    for i in range(I):
        scale = true_a[i]*true_theta[p] - true_b[i]
        true_W[p][i] = torch.from_numpy(pm.draw(PG.dist(h=1, z=scale)))
        
# exam data:
Y = torch.empty(size=(P, I))

for i in range(I):
    #print(i)
    for p in range(P):
        #print(p)
        prob = torch.exp(true_a[i]*true_theta[p] - true_b[i]) / (1 + torch.exp(true_a[i]*true_theta[p] - true_b[i]))
        B = torch.distributions.Bernoulli(prob)
        Y[p][i] = B.sample()

print(Y)

tensor([[1., 0., 0., 0., 1., 0., 1., 0., 0., 1.],
        [1., 1., 0., 1., 1., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 1., 1., 1., 0., 1., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 1., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1., 1., 1., 1.],
        [0., 0., 1., 0., 1., 1., 1., 1., 0., 0.],
        [1., 0., 0., 1., 1., 0., 1., 1., 0., 1.],
        [0., 1., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 1., 0., 0., 0., 1., 0., 1., 0.],
        [0., 0., 0., 0., 1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 0., 1., 1., 1., 0., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 1., 1., 0.],


In [9]:
samples = polyagamma(init_a, init_b, init_t, init_W, Y, sigmasq_a, sigmasq_b, sigmasq_t, niter=100)

Starting Gibbs sampler for Polya-Gamma... 
--------------------------------------------



  1%|          | 1/100 [00:06<10:35,  6.42s/it]

1.0
1.1502503027281782e+30
1.0
-1.8488902076008173e-10
1.0
1.989342166010466e+20
1.0
1.7561342247063294e-07
1.0
-1.7618344827496912e-07
1.0
-1.762647485747948e-07
1.0
-1.7744716274137318e-07
1.0
-1.801071505269647e-07
1.0
-4.786611107476801e-10
1.0
1.7680243047379918e-07


  1%|          | 1/100 [00:12<21:09, 12.82s/it]

0.0
nan


ValueError: Expected parameter loc (Tensor of shape ()) of distribution Normal(loc: nan, scale: 0.0) to satisfy the constraint Real(), but found invalid values:
nan